In [1]:
import pandas as pd
import math
import numpy as np
import json

from pathlib import Path
import scipy.stats as st

In [2]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "val_log.csv").is_file():
        with open(run / 'config.json') as f:
            cfg = json.load(f)

        num_epochs = cfg['num_epochs']
        val_iter = cfg['validate_iter']
        num_epochs = int(num_epochs / val_iter)

        valid_log = pd.read_csv(run / "val_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-1) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("val_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def collect_one(model_name, run, csv_file):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    
    with open(run / 'config.json') as f:
        cfg = json.load(f)

    run_number = cfg['seed']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = int(run_number)
    data['inv_temp'] = int(inv_temp)
    data['regime'] = int(regime)
    
    return data

def collect_all(model_name, root, csv_file, regimes=['1', '2', '5', '10', '20','100'], ignore_outliers=True):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            if regime.name.rsplit("-", 1)[1] in regimes:
                for run in list(regime.glob("run-*")):
                    if ignore_outliers and 'outlier' in run.as_posix():
                        continue
                    else:
                        metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [3]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], int(model_group[1]), int(model_group[2]), int(model_group[3])
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90, return_ranges=False):

    def compute_ci(values, return_ranges=False):
        ci = st.t.interval(confidence_level, len(values)-1, loc=np.mean(values), scale=np.std(values)/(len(values)**0.5))

        if return_ranges:
            return ci
        else:
            return (ci[1]-ci[0]) / 2

    #mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', 'mean'), ("CI {}%".format(confidence_level), compute_ci)]) 

    return mean_metrics

<h1>Atrial Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [4]:
EXP_ROOT = "./runs"

REGIMES = ['100']

runs = {
    'H-UNet3D-SWTA-T': list(Path(EXP_ROOT + '/Atrial/hebbian_unsup/').glob('unet3d_swta_t')),
}

In [5]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [6]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'])

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet3D-SWTA-T,0,1,100,34.85,21.10
1,H-UNet3D-SWTA-T,0,5,100,21.15,11.83
2,H-UNet3D-SWTA-T,0,10,100,22.57,12.72
3,H-UNet3D-SWTA-T,0,20,100,25.66,14.72
4,H-UNet3D-SWTA-T,0,50,100,27.22,15.76
5,H-UNet3D-SWTA-T,0,75,100,20.24,11.26
6,H-UNet3D-SWTA-T,0,100,100,20.21,11.24


/home/luca/anaconda3/envs/xnet/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2244: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/luca/anaconda3/envs/xnet/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2245: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


Dice         Jaccard        
                                  Mean CI 0.9%    Mean CI 0.9%
Model           Inv Temp Regime                               
H-UNet3D-SWTA-T 1        100     34.85     NaN   21.10     NaN
                5        100     21.15     NaN   11.83     NaN
                10       100     22.57     NaN   12.72     NaN
                20       100     25.66     NaN   14.72     NaN
                50       100     27.22     NaN   15.76     NaN
                75       100     20.24     NaN   11.26     NaN
                100      100     20.21     NaN   11.24     NaN

In [4]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    'H-UNet3D-SWTA-T': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('h_unet3d_swta_t')),
}

In [6]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [7]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet3D-SWTA-T,0,1,1,36.85,22.59
1,H-UNet3D-SWTA-T,0,1,2,38.03,23.48
2,H-UNet3D-SWTA-T,0,1,5,40.47,25.37
3,H-UNet3D-SWTA-T,0,1,10,40.88,25.69
4,H-UNet3D-SWTA-T,0,1,20,42.63,27.09
...,...,...,...,...,...,...
170,H-UNet3D-SWTA-T,4,100,1,60.54,43.41
171,H-UNet3D-SWTA-T,4,100,2,62.79,45.76
172,H-UNet3D-SWTA-T,4,100,5,79.29,65.68
173,H-UNet3D-SWTA-T,4,100,10,86.21,75.75


Dice           Jaccard          
                                   Mean   CI 0.9%    Mean   CI 0.9%
Model           Inv Temp Regime                                    
H-UNet3D-SWTA-T 1        1       34.342  4.932441  20.848  3.450192
                         2       38.304  1.767018  23.706  1.339093
                         5       38.098  1.525895  23.544  1.166206
                         10      39.846  1.371685  24.890  1.068274
                         20      43.866  1.379245  28.108  1.136455
                5        1       29.796  2.989640  17.548  2.100071
                         2       39.262  2.562089  24.462  2.003731
                         5       40.656  1.736571  25.532  1.376710
                         10      42.814  3.604083  27.316  2.950258
                         20      46.264  3.395537  30.168  2.902466
                10       1       29.778  3.360707  17.548  2.331055
                         2       39.434  2.914294  24.608  2.266404
                         5       43.212  1.715885  27.578  1.389512
                         10      46.274  1.300274  30.112  1.103719
                         20      51.094  1.305841  34.326  1.170096
                20       1       49.674  3.411129  33.122  3.092218
                         2       57.514  7.072098  40.738  6.839821
                         5       71.042  2.811681  55.172  3.415814
                         10      78.326  5.589439  64.738  7.313191
                         20      83.168  2.773141  71.294  4.189428
                50       1       59.360  4.694569  42.384  4.825200
                         2       74.066  4.689189  59.040  5.665453
                         5       81.860  2.010475  69.342  2.886073
                         10      84.926  1.961509  73.852  2.955969
                         20      88.598  0.496181  79.528  0.800630
                75       1       54.088  8.417113  37.566  7.868266
                         2       71.544  3.262646  55.804  3.902272
                         5       81.634  2.091789  69.024  2.969304
                         10      84.870  2.431719  73.800  3.586894
                         20      88.120  0.354272  78.764  0.561792
                100      1       57.708  6.241900  40.846  6.031270
                         2       72.876  5.313084  57.618  6.351602
                         5       80.476  2.976282  67.438  4.087677
                         10      84.770  2.714584  73.664  3.980905
                         20      88.264  0.359902  78.998  0.576511

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset.</p>

In [12]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    #'UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('unet3d')),
    #'VNet': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('vnet')),
    #'EM-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('em_unet3d')),
    #'CPS-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('cps_unet3d')),
    #'UAMT-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('uamt_unet3d')),
    #'URPC-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('urpc_unet3d')),
    'CCT-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('cct_unet3d')),
    #'DTC-UNet3D': list(Path(EXP_ROOT + '/Atrial/semi_sup/').glob('dtc_unet3d')),
}

In [13]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [14]:
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,CCT-UNet3D,0,1,1,64.70,47.82
1,CCT-UNet3D,0,1,2,78.48,64.59
2,CCT-UNet3D,0,1,5,85.54,74.73
3,CCT-UNet3D,0,1,10,87.20,77.29
4,CCT-UNet3D,0,1,20,90.93,83.36
5,CCT-UNet3D,1,1,1,56.06,38.95
6,CCT-UNet3D,1,1,2,74.89,59.86
7,CCT-UNet3D,1,1,5,84.89,73.75
8,CCT-UNet3D,1,1,10,89.04,80.25
9,CCT-UNet3D,1,1,20,91.02,83.52


Dice           Jaccard          
                              Mean   CI 0.9%    Mean   CI 0.9%
Model      Inv Temp Regime                                    
CCT-UNet3D 1        1       62.932  6.055446  46.226  6.404382
                    2       78.636  2.741297  64.886  3.720798
                    5       85.110  0.497776  74.080  0.753239
                    10      88.802  1.070474  79.872  1.739779
                    20      90.742  0.522590  83.052  0.867227